<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from timeit import default_timer as timer
t1 = timer()
try:
  from google.colab import drive
  drive.mount("/content/drive/", force_remount = True)
  import torch, torchvision, os
  from torch import nn, optim
  from torch.nn import CrossEntropyLoss
  from torch.utils.data import DataLoader
  from torchvision import datasets, transforms
  from PIL import Image
  import matplotlib.pyplot as plt
  from tqdm.auto import tqdm
  print(f">>>> You are on ColaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)}: {e}\n>>>> Please correct {type(e)} and reload your drive")

def mytimer(t: float = timer())->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h}, mins: {m:>02}, secs: {s:>05.2f}"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
BATCH_SIZE = 64 if device == torch.device("cuda") else 32
print(f">>>> Available device: {device}")
!nvidia-smi
print(f"\n>>>> Time elapsed:\t {mytimer(timer() - t1)}")

Mounted at /content/drive/
>>>> You are on ColaB with torch version: 2.0.0+cu118
>>>> Available device: cuda
Fri Apr  7 19:01:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---

In [2]:
def parsCounter(model: nn.Module)->int:
  return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [3]:
VGG19_pretrained = torchvision.models.vgg19(weights = True)
print(VGG19_pretrained)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [4]:
print(f">>>> Total number of learnable parameters is in {VGG19_pretrained.__class__.__name__}: \
{parsCounter(VGG19_pretrained):,}")

>>>> Total number of learnable parameters is in VGG: 143,667,240


In [5]:
class AvgPool_same(nn.Module):

  def __init__(self):
    super().__init__()
    
  
  def forward(self, x: torch.Tensor)->torch.Tensor:
    return x


In [6]:
FC_Net = nn.Sequential(
    nn.Linear(in_features = 512, out_features = 256),
    nn.BatchNorm1d(num_features = 256),
    nn.ReLU(),
    nn.Dropout(p = 0.45),
    nn.Linear(in_features = 256, out_features = 128),
    nn.ReLU(),
    nn.Linear(in_features = 128, out_features = 10)
)

In [7]:
VGG19_pretrained.avgpool = AvgPool_same()
VGG19_pretrained.classifier = FC_Net
#print(VGG19_pretrained)

In [8]:
print(f">>>> Total number of trainable parameters in the transfer VGG19-model:\
{parsCounter(VGG19_pretrained):,} parameters")

>>>> Total number of trainable parameters in the transfer VGG19-model:20,190,410 parameters


In [9]:
def metrics(logits: torch.Tensor, labels: torch.Tensor)->float:
  if len(labels.unique()) > 2:
    preds = torch.softmax(logits, dim = 1).argmax(dim = 1)
  else:
    preds = torch.round(torch.sigmoid(logits))
  acc = torch.eq(preds, labels).sum().item()
  acc /= len(labels)
  acc *= 100
  return acc

In [10]:
LR = 1e-4
optimizer = optim.Adam(params = VGG19_pretrained.parameters(), lr = LR)
criterion = CrossEntropyLoss()


In [11]:
def validation(model: nn.Module,
               val_loader: DataLoader,
               criterion: nn.Module,
               EPOCHS: int = 100
               )->None:

  total_val_loss = 0
  n_samples = 0
  model = model.to(device = device)
  model.eval()
  with torch.inference_mode():
    for epoch in tqdm(range(EPOCHS + 1)):
      for idx, (data, labels) in enumerate(val_loader):
        data = data.repeat(1, 3, 1, 1).to(device = device) # Cast to RGB images
        labels = labels.to(device = device)
        logits = model(data)
        val_loss = criterion(logits, labels)
        total_val_loss += val_loss
        val_acc = metrics(logits = logits, labels = labels)
        n_samples += idx

    if epoch % 10 == 0:
      print(f">>>> End of epoch: {epoch + 1 if epoch == 0 else epoch}:\
      \tValid Loss: {total_val_loss / n_samples:.4f}:\
      \tValid Accuracy: {val_acc:.2f} %")


In [12]:
def trainer(model: nn.Module,
            train_loader: DataLoader,
            criterion: nn.Module,
            optimizer: optim,
            EPOCHS: int = 100)->None:
  
  total_tr_loss = 0
  n_samples = 0

  model = model.to(device = device)
  model.train()

  for epoch in tqdm(range(EPOCHS)):
    for idx, (data, labels) in enumerate(train_loader):
      data = data.repeat(1, 3, 1, 1).to(device = device) # cast to RGB
      labels = labels.to(device)

      logits = model(data)
      tr_loss = criterion(logits, labels)
      tr_acc = metrics(logits = logits, labels = labels)
      optimizer.zero_grad()
      tr_loss.backward()
      optimizer.step()
      total_tr_loss += tr_loss
      n_samples += idx

    if epoch % 10 == 0:
      print(f">>>> End of epoch: {epoch + 1 if epoch == 0 else epoch}:\
      \tTrain Loss: {total_tr_loss / n_samples:.4f}:\
      \tTrain accuracy: {tr_acc:.2f} %")

In [13]:
train_data = datasets.FashionMNIST(root = "/train/data",
                                   train = True,
                                   transform = transforms.Compose([
                                       transforms.Resize((32, 32)),
                                       transforms.ToTensor()
                                   ]),
                                   download = True)
val_data = datasets.FashionMNIST(root = "/test/data", train = False,
                                 transform = transforms.Compose(
                                     [transforms.Resize((32, 32)),
                                      transforms.ToTensor()]
                                 ), download = True)
train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE,
                          shuffle = True)
val_loader = DataLoader(dataset = val_data, batch_size = BATCH_SIZE, shuffle = False)

x_train, y_train = next(iter(train_loader))
x_test, y_test = next(iter(val_loader))
assert x_train.shape == x_test.shape == (BATCH_SIZE, 1, 32, 32)
assert y_train.shape == y_test.shape == (BATCH_SIZE,)

In [14]:
tic = timer()
EPOCHS = 100
print(f">>>> Start Training the {VGG19_pretrained.__class__.__name__} pretrained for {EPOCHS} epochs\
ALL PARAMETERS\n>>>> Please wait...........................................................................")
trainer(model = VGG19_pretrained,
        train_loader = train_loader,
        criterion = criterion,
        optimizer = optimizer,
        EPOCHS = EPOCHS)
print(f"\n>>>> Evaluation: Please wait.......................................................................")
validation(model = VGG19_pretrained,
           val_loader = val_loader,
           criterion = criterion,
           EPOCHS = EPOCHS)
print(f"\n>>>> End of experiment: Total Time elapsed: {mytimer(timer() - tic)}")

>>>> Start Training the VGG pretrained for 100 epochsALL PARAMETERS
>>>> Please wait...........................................................................


  0%|          | 0/100 [00:00<?, ?it/s]

>>>> End of epoch: 1:      	Train Loss: 0.0009:      	Train accuracy: 93.75 %
>>>> End of epoch: 10:      	Train Loss: 0.0003:      	Train accuracy: 93.75 %
>>>> End of epoch: 20:      	Train Loss: 0.0002:      	Train accuracy: 90.62 %
>>>> End of epoch: 30:      	Train Loss: 0.0001:      	Train accuracy: 100.00 %
>>>> End of epoch: 40:      	Train Loss: 0.0001:      	Train accuracy: 96.88 %
>>>> End of epoch: 50:      	Train Loss: 0.0001:      	Train accuracy: 100.00 %
>>>> End of epoch: 60:      	Train Loss: 0.0001:      	Train accuracy: 100.00 %
>>>> End of epoch: 70:      	Train Loss: 0.0001:      	Train accuracy: 100.00 %
>>>> End of epoch: 80:      	Train Loss: 0.0001:      	Train accuracy: 100.00 %
>>>> End of epoch: 90:      	Train Loss: 0.0000:      	Train accuracy: 100.00 %

>>>> Evaluation: Please wait.......................................................................


  0%|          | 0/101 [00:00<?, ?it/s]

>>>> End of epoch: 100:      	Valid Loss: 0.0049:      	Valid Accuracy: 100.00 %

>>>> End of experiment: Total Time elapsed: hrs: 1, mins: 06, secs: 06.00
